In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [10]:
ls data

 Volume in drive E has no label.
 Volume Serial Number is 3AE1-9EE0

 Directory of E:\Code\Datadriven\data

03-05-2018  15:32    <DIR>          .
03-05-2018  15:32    <DIR>          ..
03-05-2018  15:34    <DIR>          .ipynb_checkpoints
01-05-2018  10:19           324,512 SubmissionFormat.csv
01-05-2018  10:19         5,016,337 testx.csv
01-05-2018  10:19        20,069,199 trainx.csv
01-05-2018  10:19         1,148,327 trainy.csv
               4 File(s)     26,558,375 bytes
               3 Dir(s)  275,238,252,544 bytes free


In [6]:
PATH = 'data/'

In [33]:
trainx = pd.read_csv(f'{PATH}trainx.csv')
trainy = pd.read_csv(f'{PATH}trainy.csv')
test = pd.read_csv(f'{PATH}testx.csv')

In [34]:
df_raw = pd.concat([trainx,test],axis=0)

In [35]:
add_datepart(df_raw, 'date_recorded')

In [36]:
train_cats(df_raw)

In [37]:
def func(data):
    if data=='functional':
        return 0
    elif data=='non functional':
        return 1 
    elif data=='functional needs repair':
        return 2

In [38]:
y = trainy.status_group.apply(func)

In [39]:
z = pd.DataFrame(np.zeros((14850,)))
y = pd.concat([y,z],axis=0)

In [40]:
df_raw['status_group'] = y.astype(int)

In [41]:
df_raw.shape

(74250, 53)

In [42]:
imp = ['quantity','longitude','latitude','waterpoint_type','id','wpt_name','subvillage','construction_year',
 'gps_height','ward','funder','date_recordedElapsed','date_recordedDayofyear','extraction_type_class','population',
 'date_recordedDay','amount_tsh','lga','installer', 'scheme_name','date_recordedDayofweek', 'payment','source',
 'extraction_type','region','district_code','management','payment_type','region_code','status_group']

In [43]:
df_ = df_raw[imp]

In [112]:
for v in contin_vars:
    df_[v] = df_[v].astype('float32')
df_.dtypes

C:\Users\USER\Anaconda2\envs\fastai-cpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


quantity                  category
longitude                  float32
latitude                   float32
waterpoint_type           category
id                         float32
wpt_name                  category
subvillage                category
construction_year          float32
gps_height                 float32
ward                      category
funder                    category
date_recordedElapsed       float32
date_recordedDayofyear     float32
extraction_type_class     category
population                 float32
date_recordedDay           float32
amount_tsh                 float32
lga                       category
installer                 category
scheme_name               category
date_recordedDayofweek     float32
payment                   category
source                    category
extraction_type           category
region                    category
district_code              float32
management                category
payment_type              category
region_code         

In [113]:
df,y,nas,mapper = proc_df(df_,'status_group',do_scale=True)

C:\Users\USER\Anaconda2\envs\fastai-cpu\lib\site-packages\fastai\structured.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.drop(ignore_flds, axis=1, inplace=True)


In [114]:
df.dtypes

quantity                     int8
longitude                 float64
latitude                  float64
waterpoint_type              int8
id                        float64
wpt_name                    int32
subvillage                  int16
construction_year         float64
gps_height                float64
ward                        int16
funder                      int16
date_recordedElapsed      float64
date_recordedDayofyear    float64
extraction_type_class        int8
population                float64
date_recordedDay          float64
amount_tsh                float64
lga                          int8
installer                   int16
scheme_name                 int16
date_recordedDayofweek    float64
payment                      int8
source                       int8
extraction_type              int8
region                       int8
district_code             float64
management                   int8
payment_type                 int8
region_code               float64
dtype: object

In [64]:
cat_vars=[]
contin_vars=[]
for k in df:
    try:
        if trainx[k].dtype == trainx.region.dtype:
            cat_vars.append(k)
        elif trainx[k].dtype != trainx.region.dtype:
            contin_vars.append(k)
    except KeyError:
        contin_vars.append(k)
        print(k)
        continue


date_recordedElapsed
date_recordedDayofyear
date_recordedDay
date_recordedDayofweek


## Deep Learning

In [115]:
def split_vals(a,n):
    return a[:n].copy(), a[n:].copy()

In [116]:
n_valid = 14850 # same as the size of test set
n_trn = len(df)- n_valid
x_train, x_test = split_vals(df,n_trn)
y_train, y_test = split_vals(y, n_trn)

In [117]:
x_train.dtypes

quantity                     int8
longitude                 float64
latitude                  float64
waterpoint_type              int8
id                        float64
wpt_name                    int32
subvillage                  int16
construction_year         float64
gps_height                float64
ward                        int16
funder                      int16
date_recordedElapsed      float64
date_recordedDayofyear    float64
extraction_type_class        int8
population                float64
date_recordedDay          float64
amount_tsh                float64
lga                          int8
installer                   int16
scheme_name                 int16
date_recordedDayofweek    float64
payment                      int8
source                       int8
extraction_type              int8
region                       int8
district_code             float64
management                   int8
payment_type                 int8
region_code               float64
dtype: object

In [77]:
val_idx = get_cv_idxs(59400);len(idxs)

11880

In [150]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, x_train, y_train.astype(np.float32), cat_flds=cat_vars, bs=64)

In [119]:
cat_sz = [(c, len(trainx[c].cat.categories)+1) for c in cat_vars]
cat_sz

[('quantity', 6),
 ('waterpoint_type', 8),
 ('wpt_name', 37401),
 ('subvillage', 19288),
 ('ward', 2093),
 ('funder', 1898),
 ('extraction_type_class', 8),
 ('lga', 126),
 ('installer', 2146),
 ('scheme_name', 2697),
 ('payment', 8),
 ('source', 11),
 ('extraction_type', 19),
 ('region', 22),
 ('management', 13),
 ('payment_type', 8)]

In [120]:
emb_szs = [(c, min(20, (c+1)//2)) for _,c in cat_sz];emb_szs

[(6, 3),
 (8, 4),
 (37401, 20),
 (19288, 20),
 (2093, 20),
 (1898, 20),
 (8, 4),
 (126, 20),
 (2146, 20),
 (2697, 20),
 (8, 4),
 (11, 6),
 (19, 10),
 (22, 11),
 (13, 7),
 (8, 4)]

In [151]:
y_range = (0, 3)
m = md.get_learner(emb_szs, len(x_train.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [152]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

In [154]:
m.summary

<bound method StructuredLearner.summary of MixedInputModel(
  (embs): ModuleList(
    (0): Embedding(6, 3)
    (1): Embedding(8, 4)
    (2): Embedding(37401, 20)
    (3): Embedding(19288, 20)
    (4): Embedding(2093, 20)
    (5): Embedding(1898, 20)
    (6): Embedding(8, 4)
    (7): Embedding(126, 20)
    (8): Embedding(2146, 20)
    (9): Embedding(2697, 20)
    (10): Embedding(8, 4)
    (11): Embedding(11, 6)
    (12): Embedding(19, 10)
    (13): Embedding(22, 11)
    (14): Embedding(13, 7)
    (15): Embedding(8, 4)
  )
  (lins): ModuleList(
    (0): Linear(in_features=206, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=500, bias=True)
  )
  (bns): ModuleList(
    (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True)
  )
  (outp): Linear(in_features=500, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
  (drops): ModuleList(
    (0): Dropout(p=0.001)
    (1): Dropout(p=0.01)
  

In [137]:
next(iter(md.trn_dl))

[
      2      5  38056  ...      18      8      3
      2      2  45684  ...       4      8      2
      5      5  15690  ...      14      8      7
         ...            ⋱           ...         
      3      7  27531  ...       1      8      4
      1      7   9800  ...      13      8      3
      2      5  23128  ...      12      8      3
 [torch.LongTensor of size 128x16], 
 -0.2700  0.4547 -0.3357  ...  -0.4808 -0.1690 -0.0723
  0.2122 -0.7958  1.4012  ...  -1.5060 -0.4801 -0.2436
 -0.2105  1.0801 -1.2196  ...  -0.9934 -0.0652  0.2133
           ...             ⋱             ...          
  0.3893  0.8374  0.7064  ...   1.5696 -0.3764 -0.7576
  0.6831 -1.8674 -1.4817  ...  -0.4808  5.9504  4.2684
  0.5947 -0.5139 -0.8642  ...   0.5444 -0.3764 -0.5863
 [torch.FloatTensor of size 128x13], 
     0
     0
     0
     1
     1
     2
     0
     2
     2
     0
     0
     0
     1
     1
     0
     0
     0
     0
     1
     1
     2
     1
     1
     0
     1
     2
     0
     1